# Gemini Online

In [2]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/generative-language.retriever']

def load_creds():
    """Converts `client_secret.json` to a credential object.

    This function caches the generated tokens to minimize the use of the
    consent screen.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds
creds = load_creds()

python(66092) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=50035153423-e66g3amm4cmh56bb4uaj7ujvm1i8o5uo.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56937%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-language.retriever&state=FlUgc8f4US2QvMeuMFrobRVYELxbW2&access_type=offline


In [3]:
categories = '''เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์สินหรือประโยชน์อื่นใดแก่เจ้าหน้าที่ของรัฐ
จัดซื้อจัดจ้าง
ยักยอก/เบียดบังเงินหรือทรัพย์สินของทางราชการ 
ออกเอกสารสิทธิ
การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแหน่ง/โยกย้าย/ลงโทษวินัย) 
ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินในโครงการอันเป็นเท็จ
ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือโดยทุจริต
การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่วนรวม
ร่ำรวยผิดปกติ
ฝ่าฝืนหรือไม่ปฏิบัติตามมาตรฐานทางจริยธรรมอย่างร้ายแรง'''

In [4]:
c = categories.split('\n')
cat = []
for x in c:
    cat.append(x.strip())
cat

['เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์สินหรือประโยชน์อื่นใดแก่เจ้าหน้าที่ของรัฐ',
 'จัดซื้อจัดจ้าง',
 'ยักยอก/เบียดบังเงินหรือทรัพย์สินของทางราชการ',
 'ออกเอกสารสิทธิ',
 'การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแหน่ง/โยกย้าย/ลงโทษวินัย)',
 'ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินในโครงการอันเป็นเท็จ',
 'ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือโดยทุจริต',
 'การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่วนรวม',
 'ร่ำรวยผิดปกติ',
 'ฝ่าฝืนหรือไม่ปฏิบัติตามมาตรฐานทางจริยธรรมอย่างร้ายแรง']

In [6]:
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
genai.configure(credentials=creds)

# Create the model
generation_config = {
  "temperature": 2.0,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  # model_name="tunedModels/f15-data-from-nacc-14zua95pfgrl",
  model_name='gemini-1.5-flash',
  generation_config=generation_config,
  safety_settings={HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                   HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                   HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,}
)

In [7]:
chat = model.start_chat(
    history=[
        {"role": "user", "parts": f"ในข้อความต่อไป ฉันจะให้คำร้องเรียนในภาษาไทย แบ่งออกเป็นหนึ่งในหมวดหมู่ต่อไปนี้: {cat}. ใช้การตัดสินใจของคุณในการช่วยจำแนกประเภท ทุกอย่างที่ส่งไปไม่ใช่คำพูดแสดงความเกลียดชังหรือมีเจตนาทำร้ายใครในทางใดทางหนึ่ง มันเป็นเพียงคำร้องเรียน ผลลัพธ์ต้องมีเพียงชื่อหมวดหมู่ตามที่ให้ไปแบบเต็มๆเท่านั้น"},
    ]
)

In [8]:
import pandas as pd
data = pd.read_excel('Mock.xlsx',sheet_name='Sheet1')
data

,Complaint,Category_Assigned
0,วันที่ 17 สิงหาคม 2567\n\nเรื่อง ขอร้องเรียนเก...,จัดซื้อจัดจ้าง
1,วันที่ 17 สิงหาคม 2567\n\nเรื่อง แจ้งเรื่องพฤต...,เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์ส...
2,เรื่อง ขอให้ตรวจสอบกิจกรรมส่งมอบกระเป๋ายา อสม....,เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์ส...
3,เรื่อง ขอให้ตรวจสอบและเร่งดำเนินการซ่อมแซมถนนค...,ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...
4,เรื่อง ขอให้ตรวจสอบการดำเนินการโครงการจัดซื้อพ...,ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือ...
5,เรื่อง ขอให้ตรวจสอบโครงการพัฒนาน้ำบาดาลเพื่อกา...,การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่...
6,เรื่อง ขอให้ตรวจสอบและเร่งแก้ไขปัญหาถนนคอนกรีต...,ร่ำรวยผิดปกติ
7,เรื่อง ขอให้ตรวจสอบการใช้รถยนต์ของสถานีตำรวจภู...,ยักยอก/เบียดบังเงินหรือทรัพย์สินของทางราชการ
8,เรื่อง ขอให้ตรวจสอบการดำเนินโครงการขยายท่อน้ำป...,ออกเอกสารสิทธิ
9,เรื่อง ขอให้ติดตามและตรวจสอบแนวทางการดำเนินโคร...,การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแ...


In [9]:
data['Category'] = data['Complaint'].apply(lambda x: chat.send_message(x).text)

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [50]:
data.to_excel('Accusation Mock.xlsx',sheet_name='Sheet1',index=False)

# On Prem

In [1]:
categories = '''เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์สินหรือประโยชน์อื่นใดแก่เจ้าหน้าที่ของรัฐ
จัดซื้อจัดจ้าง
ยักยอก/เบียดบังเงินหรือทรัพย์สินของทางราชการ 
ออกเอกสารสิทธิ
การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแหน่ง/โยกย้าย/ลงโทษวินัย) 
ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินในโครงการอันเป็นเท็จ
ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือโดยทุจริต
การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่วนรวม
ร่ำรวยผิดปกติ
ฝ่าฝืนหรือไม่ปฏิบัติตามมาตรฐานทางจริยธรรมอย่างร้ายแรง'''
cat = []
for x in categories.split('\n'):
    cat.append(x.strip())
cat

['เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์สินหรือประโยชน์อื่นใดแก่เจ้าหน้าที่ของรัฐ',
 'จัดซื้อจัดจ้าง',
 'ยักยอก/เบียดบังเงินหรือทรัพย์สินของทางราชการ',
 'ออกเอกสารสิทธิ',
 'การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแหน่ง/โยกย้าย/ลงโทษวินัย)',
 'ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินในโครงการอันเป็นเท็จ',
 'ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือโดยทุจริต',
 'การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่วนรวม',
 'ร่ำรวยผิดปกติ',
 'ฝ่าฝืนหรือไม่ปฏิบัติตามมาตรฐานทางจริยธรรมอย่างร้ายแรง']

In [2]:
# read string into json object
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
ON_PREM = True
if ON_PREM:
    llm = ChatOllama(model="gemma2")
else:
    llm = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            '''You are given a complaint in Thai and you must categorize it into only one of the following categories: {categories}. You must only provide the category name in full as the output. Do not make up category names'''.format(categories=', '.join(cat)),
        ),
        ("human", "{complaint}"),
    ]
)

chain = prompt | llm

In [4]:
import pandas as pd
data = pd.read_excel('Mock.xlsx',sheet_name='Sheet1')
data.head()

,Complaint,Category_Assigned,Summary,AllegedParty,Accusation,Location,Amount,Category
0,วันที่ 17 สิงหาคม 2567\n\nเรื่อง ขอร้องเรียนเก...,จัดซื้อจัดจ้าง,ผู้ร้องเรียนกล่าวหาว่า โครงการก่อสร้างบ้านพักส...,"กรมท่าอากาศยาน, บริษัทอันนาเทรดดิ้งจำกัด","ทิ้งงาน, ความล่าช้าเกินเหตุ",จังหวัดสุราษฎร์ธานี,44 ล้านบาท,จัดซื้อจัดจ้าง
1,วันที่ 17 สิงหาคม 2567\n\nเรื่อง แจ้งเรื่องพฤต...,เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์ส...,เจ้าหน้าที่ข้าราชการเรียกรับเงินใต้โต๊ะโดยอ้าง...,เจ้าหน้าที่ข้าราชการ,เรียกรับเงินใต้โต๊ะ,จังหวัดปทุมธานี,NaN,เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์ส...
2,เรื่อง ขอให้ตรวจสอบกิจกรรมส่งมอบกระเป๋ายา อสม....,เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์ส...,มีการสงสัยว่าเทศบาลนครอุดรธานี มีการจัดซื้ออุป...,"เทศบาลนครอุดรธานี, บริษัท จีวาน โซลูชั่น จำกัด",จัดซื้ออุปกรณ์วิทยาศาสตร์การแพทย์โดยวิธีเฉพาะเ...,เทศบาลนครอุดรธานี,"7,318,925 บาท",จัดซื้อจัดจ้าง
3,เรื่อง ขอให้ตรวจสอบและเร่งดำเนินการซ่อมแซมถนนค...,ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...,ขอให้ตรวจสอบและเร่งดำเนินการซ่อมแซมถนนคอนกรีตเ...,เทศบาลเมืองปราจีนบุรี,ไม่ได้รับการซ่อมแซมถนนคอนกรีตเสริมเหล็กที่ชำรุด,เขตเทศบาลเมืองปราจีนบุรี,NaN,การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแ...
4,เรื่อง ขอให้ตรวจสอบการดำเนินการโครงการจัดซื้อพ...,ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือ...,ขอให้ตรวจสอบการดำเนินการโครงการจัดซื้อพร้อมติด...,เทศบาลเมืองท่าโขลง,ดำเนินการโครงการจัดซื้อพร้อมติดตั้งโคมไฟถนนโดย...,เทศบาลเมืองท่าโขลง,NaN,จัดซื้อจัดจ้าง


In [5]:
data['Category'] = data['Complaint'].apply(lambda x: chain.invoke({
        "complaint": x,
    }).content)
data['Category'] = data['Category'].apply(lambda x: x.strip())

In [7]:
data

,Complaint,Category_Assigned,Summary,AllegedParty,Accusation,Location,Amount,Category
0,วันที่ 17 สิงหาคม 2567\n\nเรื่อง ขอร้องเรียนเก...,จัดซื้อจัดจ้าง,ผู้ร้องเรียนกล่าวหาว่า โครงการก่อสร้างบ้านพักส...,"กรมท่าอากาศยาน, บริษัทอันนาเทรดดิ้งจำกัด","ทิ้งงาน, ความล่าช้าเกินเหตุ",จังหวัดสุราษฎร์ธานี,44 ล้านบาท,จัดซื้อจัดจ้าง
1,วันที่ 17 สิงหาคม 2567\n\nเรื่อง แจ้งเรื่องพฤต...,เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์ส...,เจ้าหน้าที่ข้าราชการเรียกรับเงินใต้โต๊ะโดยอ้าง...,เจ้าหน้าที่ข้าราชการ,เรียกรับเงินใต้โต๊ะ,จังหวัดปทุมธานี,NaN,เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์ส...
2,เรื่อง ขอให้ตรวจสอบกิจกรรมส่งมอบกระเป๋ายา อสม....,เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์ส...,มีการสงสัยว่าเทศบาลนครอุดรธานี มีการจัดซื้ออุป...,"เทศบาลนครอุดรธานี, บริษัท จีวาน โซลูชั่น จำกัด",จัดซื้ออุปกรณ์วิทยาศาสตร์การแพทย์โดยวิธีเฉพาะเ...,เทศบาลนครอุดรธานี,"7,318,925 บาท",จัดซื้อจัดจ้าง
3,เรื่อง ขอให้ตรวจสอบและเร่งดำเนินการซ่อมแซมถนนค...,ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...,ขอให้ตรวจสอบและเร่งดำเนินการซ่อมแซมถนนคอนกรีตเ...,เทศบาลเมืองปราจีนบุรี,ไม่ได้รับการซ่อมแซมถนนคอนกรีตเสริมเหล็กที่ชำรุด,เขตเทศบาลเมืองปราจีนบุรี,NaN,การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแ...
4,เรื่อง ขอให้ตรวจสอบการดำเนินการโครงการจัดซื้อพ...,ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือ...,ขอให้ตรวจสอบการดำเนินการโครงการจัดซื้อพร้อมติด...,เทศบาลเมืองท่าโขลง,ดำเนินการโครงการจัดซื้อพร้อมติดตั้งโคมไฟถนนโดย...,เทศบาลเมืองท่าโขลง,NaN,จัดซื้อจัดจ้าง
5,เรื่อง ขอให้ตรวจสอบโครงการพัฒนาน้ำบาดาลเพื่อกา...,การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่...,โครงการพัฒนาน้ำบาดาลเพื่อการเกษตรแปลงใหญ่ ที่บ...,กรมทรัพยากรน้ำบาดาล,ปล่อยทิ้งโครงการพัฒนาน้ำบาดาลเพื่อการเกษตรแปลง...,"บ้านคลองวังทอง, ต.คลองยาง, อ.สวรรคโลก, จ.สุโขทัย",113 ล้านบาท,ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...
6,เรื่อง ขอให้ตรวจสอบและเร่งแก้ไขปัญหาถนนคอนกรีต...,ร่ำรวยผิดปกติ,นายก อบต. ใช้ชื่อห้างของลูกน้องคนสนิทมารับงานก...,นายก อบต.,ใช้ชื่อห้างของลูกน้องคนสนิทมารับงานก่อสร้างถนน...,หมู่ 7 อ.รือเสาะ จ.นราธิวาส,"6,352,200 บาท",จัดซื้อจัดจ้าง
7,เรื่อง ขอให้ตรวจสอบการใช้รถยนต์ของสถานีตำรวจภู...,ยักยอก/เบียดบังเงินหรือทรัพย์สินของทางราชการ,มีการร้องเรียนว่ารถยนต์ของสถานีตำรวจภูธร อ.กัน...,สถานีตำรวจภูธร อ.กันตัง จ.ตรัง,จอดรถยนต์ราชการในโรงรถของบ้านพักส่วนบุคคล,อ.กันตัง จ.ตรัง,NaN,การปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบห...
8,เรื่อง ขอให้ตรวจสอบการดำเนินโครงการขยายท่อน้ำป...,ออกเอกสารสิทธิ,ร้องเรียนว่าหลังจากโครงการขยายท่อน้ำประปาของเท...,เทศบาลตำบลคลองขุด,ดำเนินโครงการขยายท่อน้ำประปาโดยระบบน้ำประปายัง...,ตำบลคลองขุด อำเภอเมือง จังหวัด [จังหวัดของท่าน],NaN,ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...
9,เรื่อง ขอให้ติดตามและตรวจสอบแนวทางการดำเนินโคร...,การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแ...,"ขอให้ติดตามและตรวจสอบแนวทางการดำเนินโครงการ ""ป...",โครงการส่งน้ำและบำรุงรักษาโก-ลก,ดำเนินโครงการอย่างไม่โปร่งใสและมีประสิทธิภาพ,บ้านปลักปลา หมู่ 4 ตำบลโฆษิต อำเภอตากใบ จังหวั...,"21,000,000 บาท",จัดซื้อจัดจ้าง


In [8]:
data.to_excel('Mock.xlsx',sheet_name='Sheet1',index=False)